In [1]:
import json
import geojson
import gaia

# Todo? gaia.reset()

# Initialize Girder connection
apikey = 'LcxASyej0yUl9Pr7mgAmf3kj2a7a92SsemTln8ps'
datastore = gaia.connect(apikey=apikey)

In [2]:
# Create gaia dataset for girder-hosted file
girder_path = 'Public/DEM_bare_earth.tif'  # (~4GB)

# API option 1: create internal url
datastore_url = datastore.lookup_url(girder_path)
dataset = gaia.create(datastore_url)

# API option 2: pass in tuple as the data source
# data_source = (datastore, 'Public/DEM_bare_earth.tif')
# dataset = gaia.create(data_source)

# Todo is there a better api for these options?
dataset.opacity = 0.5
dataset.set_mapnik_style({
    'band': 1,
    'max': 5000,
    'min': 2000,
    'palette': 'matplotlib.Plasma_6',
    'scheme': 'linear',
})

scene = gaia.show(dataset)
display(scene)

tiles_url http://localhost:8989/api/v1/item/5bca474261700e00077b2220/tiles/zxy/{z}/{x}/{y}?encoding=PNG&projection=EPSG:3857&style=%7B%22palette%22%3A+%22matplotlib.Plasma_6%22%2C+%22max%22%3A+5000%2C+%22band%22%3A+1%2C+%22scheme%22%3A+%22linear%22%2C+%22min%22%3A+2000%7D


scene(center={'x': 0.0, 'y': 0.0}, layout=Layout(align_self='stretch', height='400px'))

In [3]:
import ipywidgets as widgets
z = widgets.FloatSlider(min=1, max=16, value=scene.zoom, layout=dict(width='95%'))

In [4]:
newlink = widgets.jslink((z, 'value'), (scene, 'zoom'))
display(z)

FloatSlider(value=11.086737, layout=Layout(width='95%'), max=16.0, min=1.0)

In [5]:
# Generate small crop geometry
meta = dataset.get_metadata()
bounds = meta.get('bounds',{}).get('coordinates')[0]
assert bounds, 'Dataset bounds missing'
# print()
# print(bounds)

# Compute center coordinates
x = (bounds[0][0] + bounds[2][0]) / 2.0
y = (bounds[0][1] + bounds[2][1]) / 2.0

# Use smll percentage of height & width
dx = 0.02 * (bounds[2][0] - bounds[0][0])
dy = 0.02 * (bounds[2][1] - bounds[0][1])
rect = [
    [x,y], [x, y-dy], [x-dx, y-dy], [x-dx, y], [x,y]
]
# print()
# print(rect)

# Must pass rectangle in as a LIST, in order to get geom formatted the way resgeodata uses
crop_geom = geojson.Polygon([rect])
print()
print(crop_geom)


{"coordinates": [[[-106.9669195224971, 38.94564248418757], [-106.9669195224971, 38.94200130082215], [-106.97010125138438, 38.94200130082215], [-106.97010125138438, 38.94564248418757], [-106.9669195224971, 38.94564248418757]]], "type": "Polygon"}


In [6]:
# Crop the dataset
from gaia import preprocess
cropped_dataset = preprocess.crop2(dataset, crop_geom, name='crop100m.tif')
# Todo output should say "job started" or something like that
# Todo get resonantgeodata to NOT copy source image

Deleting existing (item crop100m.tif)
 1 Checking job 5bd2262e61700e013c22a5b4 status... 2
 2 Checking job 5bd2262e61700e013c22a5b4 status... 2
 3 Checking job 5bd2262e61700e013c22a5b4 status... 2
 4 Checking job 5bd2262e61700e013c22a5b4 status... 2
 5 Checking job 5bd2262e61700e013c22a5b4 status... 2
 6 Checking job 5bd2262e61700e013c22a5b4 status... 2
 7 Checking job 5bd2262e61700e013c22a5b4 status... 2
 8 Checking job 5bd2262e61700e013c22a5b4 status... 2
 9 Checking job 5bd2262e61700e013c22a5b4 status... 2
10 Checking job 5bd2262e61700e013c22a5b4 status... 2
11 Checking job 5bd2262e61700e013c22a5b4 status... 2
12 Checking job 5bd2262e61700e013c22a5b4 status... 2
13 Checking job 5bd2262e61700e013c22a5b4 status... 3


In [7]:
# Todo handle "pending" datasets
print()
cropped_meta = cropped_dataset.get_metadata()
print('Cropped dataset width {}, height {}'.format(
    cropped_meta['width'], cropped_meta['height']))
#print(cropped_meta)


Cropped dataset width 570, height 821


In [10]:
# Todo Update gaia.show() for girder-resident datasets
#scene = gaia.show(cropped_dataset, representation="outline")
cropped_dataset.opacity = 0.5
cropped_dataset.set_mapnik_style({
    'band': 1,
    'max': 5000,
    'min': 2000,
    'palette': 'matplotlib.Plasma_6',
    'scheme': 'linear',
})
dataset.opacity= 0.1
#scene = gaia.show(cropped_dataset)
scene = gaia.show([cropped_dataset, dataset])
scene


tiles_url http://localhost:8989/api/v1/item/5bca474261700e00077b2220/tiles/zxy/{z}/{x}/{y}?encoding=PNG&projection=EPSG:3857&style=%7B%22palette%22%3A+%22matplotlib.Plasma_6%22%2C+%22max%22%3A+5000%2C+%22band%22%3A+1%2C+%22scheme%22%3A+%22linear%22%2C+%22min%22%3A+2000%7D
tiles_url http://localhost:8989/api/v1/item/5bd2262e61700e013c22a5b3/tiles/zxy/{z}/{x}/{y}?encoding=PNG&projection=EPSG:3857&style=%7B%22palette%22%3A+%22matplotlib.Plasma_6%22%2C+%22max%22%3A+5000%2C+%22band%22%3A+1%2C+%22scheme%22%3A+%22linear%22%2C+%22min%22%3A+2000%7D


scene(center={'x': 0.0, 'y': 0.0}, layout=Layout(align_self='stretch', height='400px'))

In [11]:
import ipywidgets as widgets
z = widgets.FloatSlider(min=1, max=16, value=scene.zoom, layout=dict(width='95%'))

In [12]:
newlink = widgets.jslink((z, 'value'), (scene, 'zoom'))
display(z)

FloatSlider(value=11.086737, layout=Layout(width='95%'), max=16.0, min=1.0)